# Tarefa:


### Introdução:
Considere uma cadeia unidimensional composta por N átomos conectados por molas idênticas de constante elástica k . O sistema pode ser de dois tipos:
* Cadeia Ternária: As massas alternam entre m e m2=3m e m3=5m  ao longo da cadeia.
* Cadeia Homogênea: Todas as massas são iguais a m .
Ambos os sistemas devem ser analisados sob condições de contorno com pontas livres (as extremidades da cadeia não estão presas).
 
Parâmetros:
* Massas: m= 1kg , 
* Constante elástica:  k = 1N/m
* Tamanhos da cadeia: N = 100, 1000 e 10.000
 
### Casos
1. Cálculo das Frequências de Vibração
    * Para cada valor de N , calcule as frequências naturais de vibração para:
    * Cadeia ternária(alternância  --m--m2--m3--m--m2--m3-- )
    * Cadeia homogênea (todas as massas iguais)
    * Considere sempre as pontas livres.
2. Densidade de Estados
    * Construa histogramas (densidade de estados) para os valores das frequências em cada caso.
    * Compare os histogramas entre a cadeia binária e a homogênea, para cada tamanho de N .
3. Deslocamentos Relativos (Modos Normais)
    * Para cada cadeia, obtenha os vetores de deslocamento (modos normais) correspondentes:
    * Às cinco menores frequências (modos de baixa energia)
    * Às cinco maiores frequências (modos de alta energia)
    * Apresente gráficos dos deslocamentos relativos dos átomos para esses modos.
4. Análise e Comparação
    * Analise como a alternância de massas (ternária) e a homogeneidade afetam:
    * A distribuição das frequências (densidade de estados)
    * Os padrões de deslocamento dos modos normais
    * Discuta a presença de possíveis lacunas de frequência (band gaps) e a localização dos modos.

### Orientações para Implementação em Python
* O código deve ser bem documentado e comentado, facilitando o entendimento de cada etapa.
* Utilize bibliotecas como `numpy` e `matplotlib` para cálculos e visualizações.
* Monte a matriz dinâmica do sistema e calcule seus autovalores e autovetores.
* Para grandes valores de N, otimize o código para eficiência computacional.
 
### Instruções para Entrega
* Crie gráficos (histogramas, modos normais, etc.)
* Certifique-se de que cada gráfico esteja devidamente identificado e relacionado a cada caso analisado.

### Sugestão de Estrutura para a resposta
1. Introdução
    * Breve explicação do modelo massa-mola, da diferença entre cadeias homogêneas e binárias, e da relevância das condições de contorno livres.
2. Metodologia
    * Descrição do método numérico utilizado para calcular frequências e modos.
    * Detalhamento da montagem da matriz dinâmica para cadeias binárias e homogêneas.
3. Resultados
    * Histogramas das densidades de estados para cada caso.
    * Gráficos dos deslocamentos relativos para os modos selecionados.
    * Tabela comparativa dos principais resultados.
4. Discussão
    * Interpretação dos efeitos da alternância de massas e das pontas livres.
    * Observações sobre lacunas de frequência, localização de modos e possíveis aplicações.
5. Conclusão
    * Síntese dos principais achados.

#### Instalando pacotes

In [5]:
pip install numpy matplotlib scipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.sparse import diags
from scipy.sparse.linalg import eigsh, lobpcg, ArpackNoConvergence


def is_hermitian(A, tol=1e-8):
    """
    Verifica se a matriz A é Hermitiana dentro da tolerância tol.
    """
    return np.allclose(A, A.T.conj(), atol=tol)


def build_dynamic_matrix_sparse(masses, k=1.0):
    """
    Monta a matriz dinâmica esparsa D para uma cadeia de pontas livres.
    """
    N = len(masses)
    diag = np.full(N, 2*k) / masses
    diag[0]  = k / masses[0]
    diag[-1] = k / masses[-1]
    off = -k / np.sqrt(masses[:-1] * masses[1:])
    D = diags([off, diag, off], offsets=[-1,0,1], format='csr')

    D = 0.5 * (D + D.T)  # simetrização
    A = D.toarray()
    if not is_hermitian(A):
        raise ValueError("Matriz D não é Hermitiana após simetrização!")
    return D, A


def compute_frequencies_and_modes(D, A, num_low=5, num_high=5, sigma=1e-6):
    """
    Calcula autovalores/auto-vetores:
      - Frequências completas (via A)
      - Modos de baixa e alta energia (via D)
    Retorna dict com freqs_full, freq_low, freq_high, phi_low, phi_high
    """
    # Frequências completas
    w2_full = np.linalg.eigvalsh(A)
    freqs_full = np.sqrt(np.clip(w2_full, 0, None))

    # modos baixa frequência (shift-invert)
    w2_low, vecs_low = eigsh(D, k=num_low, sigma=sigma, which='LM', tol=1e-8)
    freqs_low = np.sqrt(np.clip(w2_low, 0, None))
    idx0 = np.argmin(freqs_low)
    vecs_low[:, idx0] = np.ones(D.shape[0])
    freqs_low[idx0] = 0.0
    low_order = np.argsort(freqs_low)

    # modos alta frequência com fallback
    try:
        w2_high, vecs_high = eigsh(D, k=num_high, which='LM', tol=1e-6)
    except (ArpackNoConvergence, RuntimeError):
        rng = np.random.default_rng(1234)
        X0 = rng.random((D.shape[0], num_high))
        w2_high, vecs_high = lobpcg(D, X0, largest=True, tol=1e-6)
    freqs_high = np.sqrt(np.clip(w2_high, 0, None))
    high_order = np.argsort(freqs_high)[::-1]

    return {
        'freqs_full': freqs_full,
        'freq_low': freqs_low[low_order],
        'phi_low': vecs_low[:, low_order],
        'freq_high': freqs_high[high_order],
        'phi_high': vecs_high[:, high_order]
    }


def to_physical(phi_mat, masses):
    """
    Converte autovetores (φ) em deslocamentos físicos x = φ / sqrt(m).
    Normaliza cada modo.
    """
    x = phi_mat / np.sqrt(masses)[:, None]
    for j in range(x.shape[1]):
        x[:, j] /= np.linalg.norm(x[:, j])
    return x


def plot_and_save(data, masses, label, N, outdir='results'):
    """
    Gera e salva DOS e modos normais.
    """
    freqs_full = data['freqs_full']
    freq_low = data['freq_low']
    freq_high = data['freq_high']
    phi_low = data['phi_low']
    phi_high = data['phi_high']

    # DOS
    plt.figure(figsize=(6,4))
    plt.hist(freqs_full, bins=50, density=True, alpha=0.7)
    plt.title(f'DOS — {label} (N={N})')
    plt.xlabel('ω')
    plt.ylabel('Densidade')
    plt.tight_layout()
    plt.savefig(f"{outdir}/{label}_DOS_N{N}.png", dpi=150)
    plt.close()

    # Modos físicos
    low_phys = to_physical(phi_low, masses)
    high_phys = to_physical(phi_high, masses)

    # 5 modos de baixa energia
    for i in range(min(5, low_phys.shape[1])):
        plt.figure(figsize=(6,3))
        plt.plot(low_phys[:,i])
        plt.title(f'Modo Baixa Energia {label} {i+1} — N={N} (ω={freq_low[i]:.3f})')
        plt.tight_layout()
        plt.savefig(f"{outdir}/{label}_low_mode{i+1}_N{N}.png", dpi=150)
        plt.close()

    # 5 modos de alta energia
    for i in range(min(5, high_phys.shape[1])):
        plt.figure(figsize=(6,3))
        plt.plot(high_phys[:,i])
        plt.title(f'Modo Alta Energia {label} {i+1} — N={N} (ω={freq_high[i]:.3f})')
        plt.tight_layout()
        plt.savefig(f"{outdir}/{label}_high_mode{i+1}_N{N}.png", dpi=150)
        plt.close()

def main():
    k = 1.00
    Ns = [100, 1000, 10000]
    import os
    os.makedirs('results', exist_ok=True)

    for N in Ns:
        masses_h = np.ones(N)
        masses_t = np.tile([1,3,5], int(np.ceil(N/3)))[:N]

        for label, masses in [('Homogênea', masses_h), ('Ternária', masses_t)]:
            D, A = build_dynamic_matrix_sparse(masses, k)
            data = compute_frequencies_and_modes(D, A)
            plot_and_save(data, masses, label, N)

    print("Execução finalizada. Gráficos salvos em 'results/'.")

if __name__ == '__main__':
    main()


Execução finalizada. Gráficos salvos em 'results/'.
